In [ ]:
import pyDOE
from scipy import stats
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from scipy.optimize import Bounds
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from collections import namedtuple
from matplotlib import cm
from scipy.stats import mannwhitneyu
import cma
from scipy.optimize import minimize
from scipy.optimize import Bounds
import sys
from sklearn.preprocessing import MinMaxScaler
import pyDOE
import pandas as pd
import numpy as np
import scipy.stats.distributions as dist
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from sklearn.ensemble import RandomForestRegressor
from scipy.optimize import minimize
from scipy.optimize import Bounds
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.gaussian_process.kernels import DotProduct
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from collections import namedtuple
from sklearn.linear_model import ElasticNet
from RBFN import RBFN

## Meta-Models

In [3]:
''' Elastic Net Regression for Minimax Robustness'''
def elastic_net_minimax(train_data,test_data):
    scaler =  MinMaxScaler().fit(np.r_[train_data.iloc[:,:20].values, test_data.values])
    regr=ElasticNet(alpha= 0.01090686643551237   ,random_state=0 , l1_ratio= 0.46146146146146144, fit_intercept =True, max_iter=1000,selection='random'
                   ).fit(scaler.transform(train_data.iloc[:,:20]), train_data.iloc[:,21])
    pred = regr.predict(scaler.transform(test_data))
    def predict(scaler, regr):
        def __predict__(x):
            x = np.array([x[0] , x[1] , x[2], x[3], x[4], x[0] * x[1], x[0] * x[2], x[0] * x[3], x[0] * x[4],
                          x[1] * x[2] , x[1] * x[3], x[1] * x[4], x[2] * x[3], x[2] * x[4], x[3] * x[4], x[0] **2 , 
                          x[1] **2, x[2] **2 , x[3] **2, x[4] **2])
            x = np.atleast_2d(x)
            return regr.predict(scaler.transform(x))
        return __predict__
    return regr,pred, predict(scaler, regr)

''' Elastic Net Regression for Minimax Robustness'''
def elastic_net_composite(train_data,test_data):
    scaler =  MinMaxScaler().fit(np.r_[train_data.iloc[:,:20].values, test_data.values])
    regr=ElasticNet(alpha= 0.01015356284041802       ,random_state=0 , l1_ratio= 0.26126126126126126, fit_intercept =True, max_iter=1000,selection='random'
                   ).fit(scaler.transform(train_data.iloc[:,:20]), train_data.iloc[:,-1])
    pred = regr.predict(scaler.transform(test_data))
    def predict(scaler, regr):
        def __predict__(x):
            x = np.array([x[0] , x[1] , x[2], x[3], x[4], x[0] * x[1], x[0] * x[2], x[0] * x[3], x[0] * x[4],
                          x[1] * x[2] , x[1] * x[3], x[1] * x[4], x[2] * x[3], x[2] * x[4], x[3] * x[4], x[0] **2 , 
                          x[1] **2, x[2] **2 , x[3] **2, x[4] **2])
            x = np.atleast_2d(x)
            return regr.predict(scaler.transform(x))
        return __predict__
    return regr,pred, predict(scaler, regr)


''' Kriging for Robust Regularization'''
def kriging_minimax(train_data,test_data):
    kernel =  RBF( 46,(99,44818) )
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:5].values, test_data.values])
    gpr = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer=35,random_state=0,
                                   normalize_y=True ).fit(scaler.transform(train_data.iloc[:,:5]), train_data.iloc[:,6])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)


''' Kriging for Robust Composition'''
def kriging_composite(train_data,test_data):
    kernel =  RBF( 46,(99,44818) )
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:5].values, test_data.values])
    gpr = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer=35,random_state=0,
                                   normalize_y=True ).fit(scaler.transform(train_data.iloc[:,:5]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)


''' Random Forest Implementation for Minimax Robustness'''
def rf_minimax(train_data,test_data):
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:5].values, test_data.values])
    regr = RandomForestRegressor(random_state=7,n_estimators=98
                                ).fit(scaler.transform(train_data.iloc[:,:5]), train_data.iloc[:,6])
    pred = regr.predict(scaler.transform(test_data))
    def predict(scaler, regr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return regr.predict(scaler.transform(x))
        return __predict__
    return regr,pred, predict(scaler,regr)

''' Random Forest Implementation for Composite Robustness'''
def rf_composite(train_data,test_data):
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:5].values, test_data.values])
    regr = RandomForestRegressor(random_state=10,n_estimators=32
                                ).fit(scaler.transform(train_data.iloc[:,:5]), train_data.iloc[:,-1])
    pred = regr.predict(scaler.transform(test_data))
    def predict(scaler, regr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return regr.predict(scaler.transform(x))
        return __predict__
    return regr,pred, predict(scaler,regr)


''' KNN Regression Implementation for Minimax Robustness'''
def KNN_minimax(train_data,test_data):
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:5].values, test_data.values])
    regr = KNeighborsRegressor(n_neighbors=10,weights='distance',algorithm='brute',p=2
                               ).fit(scaler.transform(train_data.iloc[:,:5]), train_data.iloc[:,6])

    pred = regr.predict(scaler.transform(test_data))
    def predict(scaler, regr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return regr.predict(scaler.transform(x))
        return __predict__
    return regr,pred, predict(scaler,regr)

''' KNN Regression Implementation for Composite Robustness'''
def KNN_composite(train_data,test_data):
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:5].values, test_data.values])
    regr = KNeighborsRegressor(n_neighbors=4,weights='distance',algorithm='brute',p=2
                               ).fit(scaler.transform(train_data.iloc[:,:5]), train_data.iloc[:,-1])

    pred = regr.predict(scaler.transform(test_data))
    def predict(scaler, regr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return regr.predict(scaler.transform(x))
        return __predict__
    return regr,pred, predict(scaler,regr)

''' Support Vector Regression for Robust Regularization'''
def SVR_minimax(train_data,test_data):
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:5].values, test_data.values])
    gpr = SVR(gamma =  10.215830    ,C =  0.583392, epsilon=0.2,max_iter=1500).fit(
        scaler.transform(train_data.iloc[:,:5]), train_data.iloc[:,6])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)

''' Support Vector Regression for Robust Composition'''
def SVR_composite(train_data,test_data):
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:5].values, test_data.values])
    gpr = SVR(gamma =  10.215830 ,C = 0.583392, epsilon=0.2,max_iter=1500).fit(
        scaler.transform(train_data.iloc[:,:5]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)


''' RBF Network Interpolation for Robust Regularization'''
def RBF_minimax(train_data,test_data):
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:5].values, test_data.values])
    model = RBFN(hidden_shape=88, sigma= 0.001000)
    model.fit(scaler.transform(np.array(train_data.iloc[:,:5])), np.array(train_data.iloc[:,6]))
    pred = model.predict(scaler.transform(np.array(test_data)))
    def predict(scaler, model):
        def __predict__(x):
            x = np.atleast_2d(x)
            return model.predict(scaler.transform(x))
        return __predict__
    return model,pred, predict(scaler,model)

''' RBF Network Interpolation for Robust Composition'''
def RBF_composite(train_data,test_data):
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:5].values, test_data.values])
    model = RBFN(hidden_shape=88, sigma= 0.001000)
    model.fit(scaler.transform(np.array(train_data.iloc[:,:5])), np.array(train_data.iloc[:,-1]))
    pred = model.predict(scaler.transform(np.array(test_data)))
    def predict(scaler, model):
        def __predict__(x):
            x = np.atleast_2d(x)
            return model.predict(scaler.transform(x))
        return __predict__
    return model,pred, predict(scaler,model)

In [4]:
train = pd.read_csv(sys.path[1]+str('\\Training_Data_Sets\\250Samples.csv')).iloc[:,1:]
test = pd.read_csv(sys.path[1]+str('\\Test_Data_Sets\\375Samples.csv')).iloc[:,1:]
model_k_m,pred_k_m, predict_k_m = kriging_minimax(train,test.iloc[:,:5])
model_k_c,pred_k_c, predict_k_c = kriging_composite(train,test.iloc[:,:5])
model_r_m,pred_r_m, predict_r_m = rf_minimax(train,test.iloc[:,:5])
model_r_c,pred_r_c, predict_r_c = rf_composite(train,test.iloc[:,:5])
model_n_m,pred_n_m, predict_n_m = KNN_minimax(train,test.iloc[:,:5])
model_n_c,pred_n_c , predict_n_c= KNN_composite(train,test.iloc[:,:5])
model_s_m,pred_s_m, predict_s_m = SVR_minimax(train,test.iloc[:,:5])
model_s_c,pred_s_c, predict_s_c = SVR_composite(train,test.iloc[:,:5])
model_b_m,pred_b_m, predict_b_m = RBF_minimax(train,test.iloc[:,:5])
model_b_c,pred_b_c, predict_b_c = RBF_composite(train,test.iloc[:,:5])
train = create_basis_function_train(train)
test = create_basis_function_test(test)
model_m_m,pred_m_m, predict_m_m = elastic_net_minimax(train,test.iloc[:,:20])
model_m_c,pred_m_c, predict_m_c = elastic_net_composite(train,test.iloc[:,:20])

In [5]:
Columns = ['Benchmark', 'Kriging' , 'SVM' , 'RBFN' ,  'KNN' , 'RF'  , 'ELN' ]
Cols = []
for j in range(len(Columns)):
    for i in range(1,6):
        Cols.append(Columns[j]+'_X'+str(i))
        
const = Bounds([-32.768, -32.768 , -32.768, -32.768 , -32.768], [32.768, 32.768 , 32.768, 32.768 , 32.768])
G1,G2,G3,G4,G5 = DOE(100)
G1 = linearscaletransform(G1,range_out=(-32.768,32.768))
G2 = linearscaletransform(G2,range_out=(-32.768,32.768))
G3 = linearscaletransform(G3,range_out=(-32.768,32.768))
G4 = linearscaletransform(G4,range_out=(-32.768,32.768))
G5 = linearscaletransform(G5,range_out=(-32.768,32.768))


X_Values_RR = np.zeros([100,35])
X_Values_RC = np.zeros([100,35])
Fun_Eval_RR = np.zeros([100,7])
Fun_Eval_RC = np.zeros([100,7])

## SLSQP

In [6]:
for i in range(X_Values_RR.shape[0]):
    min_robust = minimize(robust_regularization,(np.array([G1[i],G2[i],G3[i],G4[i],G5[i]])),method='SLSQP',bounds=const)
    min_compo = minimize(composite_robustness,(np.array([G1[i],G2[i],G3[i],G4[i],G5[i]])),method='SLSQP',bounds=const)
    min_robust_k = minimize(predict_k_m,(np.array([G1[i],G2[i],G3[i],G4[i],G5[i]])),method='SLSQP',bounds=const)
    min_composite_k = minimize(predict_k_c,(np.array([G1[i],G2[i],G3[i],G4[i],G5[i]])),method='SLSQP',bounds=const)
    min_robust_s = minimize(predict_s_m,(np.array([G1[i],G2[i],G3[i],G4[i],G5[i]])),method='SLSQP',bounds=const)
    min_composite_s = minimize(predict_s_c,(np.array([G1[i],G2[i],G3[i],G4[i],G5[i]])),method='SLSQP',bounds=const)
    min_robust_b = minimize(predict_b_m,(np.array([G1[i],G2[i],G3[i],G4[i],G5[i]])),method='SLSQP',bounds=const)
    min_composite_b = minimize(predict_b_c,(np.array([G1[i],G2[i],G3[i],G4[i],G5[i]])),method='SLSQP',bounds=const)
    min_robust_n = minimize(predict_n_m,(np.array([G1[i],G2[i],G3[i],G4[i],G5[i]])),method='SLSQP',bounds=const)
    min_composite_n = minimize(predict_n_c,(np.array([G1[i],G2[i],G3[i],G4[i],G5[i]])),method='SLSQP',bounds=const)
    min_robust_r = minimize(predict_r_m,(np.array([G1[i],G2[i],G3[i],G4[i],G5[i]])),method='SLSQP',bounds=const)
    min_composite_r = minimize(predict_r_c,(np.array([G1[i],G2[i],G3[i],G4[i],G5[i]])),method='SLSQP',bounds=const)
    min_robust_m = minimize(predict_m_m,(np.array([G1[i],G2[i],G3[i],G4[i],G5[i]])),method='SLSQP',bounds=const)
    min_composite_m = minimize(predict_m_c,(np.array([G1[i],G2[i],G3[i],G4[i],G5[i]])),method='SLSQP',bounds=const)
    
   
    
    
    
    X_Values_RR [i,:] = list(min_robust.x)+list(min_robust_k.x)+list(min_robust_s.x)+list(min_robust_b.x)+list(min_robust_n.x)+list(min_robust_r.x)+list(min_robust_m.x)
    
    X_Values_RC [i,:] = list(min_compo.x)+list(min_composite_k.x)+list(min_composite_s.x)+list(min_composite_b.x)+list(min_composite_n.x)+list(min_composite_r.x)+list(min_composite_m.x)
    
    Fun_Eval_RR [i,:] = [min_robust.nfev]+[100,100,100,100,100,100]
    
    Fun_Eval_RC [i,:] = [min_compo.nfev]+[100,100,100,100,100,100]

In [7]:
X_Values_RR = pd.DataFrame(X_Values_RR)
X_Values_RR.columns = Cols
X_Values_RC = pd.DataFrame(X_Values_RC)
X_Values_RC.columns = Cols
Fun_Eval_RR = pd.DataFrame(Fun_Eval_RR)
Fun_Eval_RR.columns = Columns
Fun_Eval_RC = pd.DataFrame(Fun_Eval_RC)
Fun_Eval_RC.columns = Columns

In [8]:
X_Values_RR = pd.DataFrame(X_Values_RR)
X_Values_RR.columns = Cols
X_Values_RC = pd.DataFrame(X_Values_RC)
X_Values_RC.columns = Cols
Fun_Eval_RR = pd.DataFrame(Fun_Eval_RR)
Fun_Eval_RR.columns = Columns
Fun_Eval_RC = pd.DataFrame(Fun_Eval_RC)
Fun_Eval_RC.columns = Columns

## Meta-Modelling

In [9]:
X_Values_RR.to_csv('Results\\X_Values_RR.csv')
X_Values_RC.to_csv('Results\\X_Values_RC.csv')
Fun_Eval_RR.to_csv('Results\\Function_Eval_RR.csv')
Fun_Eval_RC.to_csv('Results\\Function_Eval_RC.csv')
RR = pd.read_csv('Results\\X_Values_RR.csv').iloc[:,1:]
RC = pd.read_csv('Results\\X_Values_RC.csv').iloc[:,1:]

In [10]:
Bench_Fun_RR = np.zeros(100)
Krig_Fun_RR = np.zeros(100)
SVM_Fun_RR = np.zeros(100)
RBFN_Fun_RR = np.zeros(100)
KNN_Fun_RR = np.zeros(100)
RF_Fun_RR = np.zeros(100)
ELN_Fun_RR = np.zeros(100)
for i in range(X_Values_RR.shape[0]):
    Bench_Fun_RR [i] = robust_regularization(X_Values_RR.iloc[i,:5])
    Krig_Fun_RR [i] = robust_regularization(X_Values_RR.iloc[i,5:10])
    SVM_Fun_RR [i] = robust_regularization(X_Values_RR.iloc[i,10:15])
    RBFN_Fun_RR [i] = robust_regularization(X_Values_RR.iloc[i,15:20])
    KNN_Fun_RR [i] = robust_regularization(X_Values_RR.iloc[i,20:25])
    RF_Fun_RR [i] = robust_regularization(X_Values_RR.iloc[i,25:30])
    ELN_Fun_RR [i] = robust_regularization(X_Values_RR.iloc[i,30:35])

In [11]:
print ('SLSQP')
print (stats.mode(Bench_Fun_RR) , np.std(Bench_Fun_RR))
print ('Kriging')
print (stats.mode(Krig_Fun_RR) , np.std(Krig_Fun_RR))
print ('SVM')
print (stats.mode(SVM_Fun_RR) , np.std(SVM_Fun_RR))
print ('RBFN')
print (stats.mode(RBFN_Fun_RR) , np.std(RBFN_Fun_RR))
print ('KNN')
print (stats.mode(KNN_Fun_RR) , np.std(KNN_Fun_RR))
print ('RF')
print (stats.mode(RF_Fun_RR) , np.std(RF_Fun_RR))
print ('ELN')
print (stats.mode(ELN_Fun_RR) , np.std(ELN_Fun_RR))

SLSQP
ModeResult(mode=array([15.55445389]), count=array([1])) 0.7460961789191021
Kriging
ModeResult(mode=array([16.45002806]), count=array([1])) 0.7934475378231729
SVM
ModeResult(mode=array([12.28576417]), count=array([1])) 0.9817394256057553
RBFN
ModeResult(mode=array([16.60293531]), count=array([1])) 0.7864565583065575
KNN
ModeResult(mode=array([11.9468831]), count=array([1])) 1.838829007005877
RF
ModeResult(mode=array([16.7693985]), count=array([1])) 0.7799088581984966
ELN
ModeResult(mode=array([21.88013274]), count=array([35])) 0.03773861148846428


In [12]:
Bench_Fun_RC = np.zeros(100)
Krig_Fun_RC = np.zeros(100)
SVM_Fun_RC = np.zeros(100)
RBFN_Fun_RC = np.zeros(100)
KNN_Fun_RC = np.zeros(100)
RF_Fun_RC = np.zeros(100)
ELN_Fun_RC = np.zeros(100)
for i in range(X_Values_RC.shape[0]):
    Bench_Fun_RC [i] = composite_robustness(X_Values_RC.iloc[i,:5])
    Krig_Fun_RC [i] = composite_robustness(X_Values_RC.iloc[i,5:10])
    SVM_Fun_RC [i] = composite_robustness(X_Values_RC.iloc[i,10:15])
    RBFN_Fun_RC [i] = composite_robustness(X_Values_RC.iloc[i,15:20])
    KNN_Fun_RC [i] = composite_robustness(X_Values_RC.iloc[i,20:25])
    RF_Fun_RC [i] = composite_robustness(X_Values_RC.iloc[i,25:30])
    ELN_Fun_RC [i] = composite_robustness(X_Values_RC.iloc[i,30:35])

In [13]:
print ('SLSQP')
print (stats.mode(Bench_Fun_RC) , np.std(Bench_Fun_RC))
print ('Kriging')
print (stats.mode(Krig_Fun_RC) , np.std(Krig_Fun_RC))
print ('SVM')
print (stats.mode(SVM_Fun_RC) , np.std(SVM_Fun_RC))
print ('RBFN')
print (stats.mode(RBFN_Fun_RC) , np.std(RBFN_Fun_RC))
print ('KNN')
print (stats.mode(KNN_Fun_RC) , np.std(KNN_Fun_RC))
print ('RF')
print (stats.mode(RF_Fun_RC) , np.std(RF_Fun_RC))
print ('ELN')
print (stats.mode(ELN_Fun_RC) , np.std(ELN_Fun_RC))

SLSQP
ModeResult(mode=array([4.63015294]), count=array([1])) 4.761625483345771
Kriging
ModeResult(mode=array([18.52140599]), count=array([1])) 0.700233240943032
SVM
ModeResult(mode=array([12.01614539]), count=array([1])) 2.1367061899670223
RBFN
ModeResult(mode=array([18.37315114]), count=array([1])) 0.698783937237862
KNN
ModeResult(mode=array([13.9933521]), count=array([1])) 1.9201264843112962
RF
ModeResult(mode=array([18.60251394]), count=array([1])) 0.6869466815214951
ELN
ModeResult(mode=array([21.70801112]), count=array([1])) 0.08354182714155828
